<a href="https://colab.research.google.com/github/Katerina2626/ML/blob/main/%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BE%D0%BB%D0%B8%D0%BC%D0%BF%D0%B8%D0%B0%D0%B4%D1%8B_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Задание

# 1.

1) Запустить ячейку ниже, чтобы получить 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста (как в примере), обучить наивный байесовский классификатор, численно оценить его работу.

In [1]:
 import random
 lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B']
 print('ваши жанры', random.choice(lst), 'и', random.choice(lst))

ваши жанры Country и R&B


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/разработка нейронных сетей/олимпиады 2.2/datasets/dataset.csv")
columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Christian') | (columns.genre == 'Pop')]

columns['lowered'] = columns['lyrics'].str.lower()
columns

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,genre,lyrics,lowered
0,Christian,"Who am I, that the Lord of all the earth Woul...","who am i, that the lord of all the earth woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...
...,...,...,...
367,Pop,Have you ever met a girl that you tried to da...,have you ever met a girl that you tried to da...
368,Pop,"Im outta luck, outta love Gotta photograph, p...","im outta luck, outta love gotta photograph, p..."
369,Pop,She was a fast machine She kept her motor cl...,she was a fast machine she kept her motor cl...
370,Pop,"Good times, these are the good times Leave yo...","good times, these are the good times leave yo..."


*Токенизация*

In [4]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Удаление стоп-слов

In [5]:
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [7]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Разделение данных

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)
columns.genre.value_counts()

Pop          100
Christian     94
Name: genre, dtype: int64

Векторизация

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [11]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [12]:
from sklearn.metrics import *
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.93      0.89      0.91        28
         Pop       0.91      0.94      0.92        31

    accuracy                           0.92        59
   macro avg       0.92      0.91      0.91        59
weighted avg       0.92      0.92      0.92        59



# 2.

2) Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

In [13]:
music = ["Lets clap, one, two, hey. Oh, yea, Let everything that has breath Praise the Lord (You got it), praise the Lord Let everything, let everything that has breath Praise the Lord, praise the Lord (Let everything) Let everything (Hey) that has breath (Hey)"]
#нижний регистр
music_lower = list(map(str.lower, music))
#токенизация
music_tok = [nltk.word_tokenize(i) for i in music_lower]
music_tokened = list(flatten(music_tok))
#удаление стоп-слов
music_withoutstop = list(set(music_tokened) - set(noise))
#лемматизация
music_pro = [lemmatizer.lemmatize(i) for i in music_withoutstop]

#векторизация
vectorized_mus = vectorizer.transform(music)
clf.predict(vectorized_mus)


array(['Christian'], dtype='<U9')

# 3.

3) С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен Дэвида Боуи от текстов песен Пола МакКартни
https://www.kaggle.com/italomarcelo/dataset-lyrics-music-mini

In [14]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
dat = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/разработка нейронных сетей/олимпиады 2.2/datasets/dataset-lyrics-musics-mini.csv")
columns = dat[['cantorNome', 'letra']]
columns = columns[(columns.cantorNome == 'david-bowie') | (columns.cantorNome == 'paul-mccartney')]

columns['lowered'] = columns['letra'].str.lower()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


*Токенизация*

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Удаление стоп-слов

In [16]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [18]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Разделение данных

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.cantorNome, train_size = 0.7)
columns.cantorNome.value_counts()

david-bowie       483
paul-mccartney    464
Name: cantorNome, dtype: int64

Векторизация

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [22]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [23]:
from sklearn.metrics import *
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

                precision    recall  f1-score   support

   david-bowie       0.79      0.79      0.79       145
paul-mccartney       0.78      0.79      0.78       140

      accuracy                           0.79       285
     macro avg       0.79      0.79      0.79       285
  weighted avg       0.79      0.79      0.79       285



# Задание
1. Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки текстов определить жанр обеих песен с помощью обученной  модели в ходе выполнения пункта `1` в Notebook `Контроль знаний 3-2.ipynb`.

Провести Анализ всез жанров песен в тестовом датасете:
- `Christian`,
- `Country`,
- `Hip-Hop`,
- `Pop`,
- `Rock`,
- `R&B`.

2. Научить модель отличать тексты песен двух певцов(выбрать самостоятельно) примере пункта `3` в Notebook `Контроль знаний 3-2.ipynb`.